### API routage

Comparaison des performances entre les différentes routes API 2 points à Lyon en vélo.
* ORS: nécessite une clé API (https://openrouteservice.org/dev/#/api-docs)
* OSRM : ne nécessite pas de clé API
* Google MAPS

In [9]:
import folium
import openrouteservice
from openrouteservice import convert
import requests

In [ ]:
# Définir deux POI (lon, lat)
poi_A = (4.85, 45.75)
poi_B = (4.83, 45.77)

##################################
#### OSR
###################################
# 

API_KEY="XXXXXXXXXXXXXXXXXXXXXYjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjJmMmYwNzU0NzVkNzQ1ODRhMGJiZjlhMWM0MzZjM2NhIiwiaCI6Im11cm11cjY0In0="


# Initialiser le client avec ta clé ORS
client = openrouteservice.Client(key=API_KEY)

# Calculer l'itinéraire (profil piéton par exemple)
route_osr = client.directions(
    coordinates=[poi_A, poi_B],
    profile='cycling-regular', #cycling-regular, foot-walking
    format='geojson'
)

# Extraire géométrie et durée
coords_osr = route_osr['features'][0]['geometry']['coordinates']
duration_osr = route_osr['features'][0]['properties']['summary']['duration'] / 60  # minutes
distance_osr = route_osr['features'][0]['properties']['summary']['distance'] / 1000  # km

print(f"Calcul avec OSR : Durée: {duration_osr:.1f} min, Distance: {distance_osr:.2f} km")


##################################
### OSRM
###################################

# Requête OSRM (profil vélo)
url = f"http://router.project-osrm.org/route/v1/cycling/{poi_A[0]},{poi_A[1]};{poi_B[0]},{poi_B[1]}?overview=full&geometries=geojson"
response = requests.get(url).json()

# Extraire géométrie et durée
route_osrm = response['routes'][0]
coords_osrm = route_osrm['geometry']['coordinates']
duration_osrm = route_osrm['duration'] / 60  # minutes
distance_osrm = route_osrm['distance'] / 1000  # km

print(f"Calcul avec OSRM : Durée: {duration_osrm:.1f} min, Distance: {distance_osrm:.2f} km")
print(f"Calcul avec Google Maps : Durée: 13 min, Distance: 3.50 km")

# Visualiser sur Folium
m = folium.Map(location=[45.76, 4.84], zoom_start=14)
folium.Marker([poi_A[1], poi_A[0]], popup="POI A").add_to(m)
folium.Marker([poi_B[1], poi_B[0]], popup="POI B").add_to(m)
folium.PolyLine([(lon, lat) for lat, lon in coords_osr], color="red", weight=2, popup=f"OSR: {duration_osr:.1f}min, {distance_osr:.2f}km").add_to(m)
folium.PolyLine([(lat, lon) for lon, lat in coords_osrm], color="blue", weight=2, popup=f"OSRM: {duration_osrm:.1f}min, {distance_osrm:.2f}km").add_to(m)

# Ajout d'une légende
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 150px; height: 90px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 padding: 10px;">
         <p style="margin:5px;"><i class="fa fa-map-marker" 
                          style="color:red"></i> OSR</p>
         <p style="margin:5px;"><i class="fa fa-map-marker" 
                          style="color:blue"></i> OSRM</p>
     </div>
     '''

m.get_root().html.add_child(folium.Element(legend_html))

m

Calcul avec OSR : Durée: 12.4 min, Distance: 3.60 km
Calcul avec OSRM : Durée: 9.3 min, Distance: 3.93 km
Calcul avec Google Maps : Durée: 13 min, Distance: 3.50 km
